# 📊 Chat Database Explorer

探索 `chat.db` 数据库中的数据

---

In [1]:
# 导入必要的库
import sqlite3
import pandas as pd
from datetime import datetime

# 数据库路径
DB_PATH = "chat.db"

def get_connection():
    """获取数据库连接"""
    return sqlite3.connect(DB_PATH)

# 测试连接
conn = get_connection()
print("✅ 数据库连接成功！")
print(f"📁 数据库路径: {DB_PATH}")

✅ 数据库连接成功！
📁 数据库路径: chat.db


## 1️⃣ 查看所有表

---

In [2]:
# 查看数据库中所有的表
query = """
SELECT name as table_name
FROM sqlite_master
WHERE type='table'
ORDER BY name;
"""

tables = pd.read_sql_query(query, conn)
print("📋 数据库中的表：")
print(tables.to_string(index=False))

📋 数据库中的表：
table_name
  messages
     users


## 2️⃣ 用户信息表 (`users`)

---

In [3]:
# 查看 users 表结构
query = """
PRAGMA table_info(users);
"""

schema = pd.read_sql_query(query, conn)
print("👤 Users 表结构：")
print(schema.to_string(index=False))

👤 Users 表结构：
 cid            name     type  notnull dflt_value  pk
   0              id  INTEGER        1       None   1
   1        username  VARCHAR        1       None   0
   2 hashed_password  VARCHAR        1       None   0
   3      created_at DATETIME        0       None   0


In [4]:
# 查询所有用户信息
query = """
SELECT 
    id,
    username,
    hashed_password,
    created_at
FROM users
ORDER BY id;
"""

users_df = pd.read_sql_query(query, conn)

print(f"👥 用户总数: {len(users_df)}")
print("\n" + "="*80)
print("👤 用户详细信息：")
print("="*80)

for idx, row in users_df.iterrows():
    print(f"\n📌 用户 {row['id']}:")
    print(f"   用户名: {row['username']}")
    print(f"   密码哈希: {row['hashed_password']}")
    print(f"   创建时间: {row['created_at']}")
    print("-"*40)

👥 用户总数: 3

👤 用户详细信息：

📌 用户 1:
   用户名: testuser
   密码哈希: $2b$12$f1ikwaoTLwwyP.Xh2.pSEOAvNyWvZVFAI3UcvxiC9bInuDE5fm0kW
   创建时间: 2026-01-20 08:21:57.353005
----------------------------------------

📌 用户 2:
   用户名: test1
   密码哈希: $2b$12$oEScRk6RfQcaAcwnJg8w9Os.SV8gZn4aBw07v.qnmCb1GTqcKG5CW
   创建时间: 2026-01-20 08:24:00.828193
----------------------------------------

📌 用户 3:
   用户名: test2
   密码哈希: $2b$12$Nmj0oy1zIPFgbhgH3j/pYuHhLqhH.WS/NTsJBrkE0u4O.vYuh4a6q
   创建时间: 2026-01-20 08:30:55.848338
----------------------------------------


In [5]:
# 表格形式展示用户信息
print("📊 用户表格视图：")
users_df

📊 用户表格视图：


,id,username,hashed_password,created_at
0,1,testuser,$2b$12$f1ikwaoTLwwyP.Xh2.pSEOAvNyWvZVFAI3Ucvxi...,2026-01-20 08:21:57.353005
1,2,test1,$2b$12$oEScRk6RfQcaAcwnJg8w9Os.SV8gZn4aBw07v.q...,2026-01-20 08:24:00.828193
2,3,test2,$2b$12$Nmj0oy1zIPFgbhgH3j/pYuHhLqhH.WS/NTsJBrk...,2026-01-20 08:30:55.848338


## 3️⃣ 聊天消息表 (`messages`)

---

In [6]:
# 查看 messages 表结构
query = """
PRAGMA table_info(messages);
"""

schema = pd.read_sql_query(query, conn)
print("💬 Messages 表结构：")
print(schema.to_string(index=False))

💬 Messages 表结构：
 cid       name     type  notnull dflt_value  pk
   0         id  INTEGER        1       None   1
   1    content  VARCHAR        1       None   0
   2    user_id  INTEGER        1       None   0
   3 created_at DATETIME        0       None   0


In [7]:
# 查询所有消息（包含用户名）
query = """
SELECT 
    m.id,
    m.content,
    m.user_id,
    u.username,
    m.created_at
FROM messages m
JOIN users u ON m.user_id = u.id
ORDER BY m.created_at DESC;
"""

messages_df = pd.read_sql_query(query, conn)

print(f"💬 消息总数: {len(messages_df)}")
print("\n" + "="*80)
print("📝 消息列表：")
print("="*80)

for idx, row in messages_df.iterrows():
    print(f"\n🆔 消息ID: {row['id']}")
    print(f"👤 用户: {row['username']} (ID: {row['user_id']})")
    print(f"💬 内容: {row['content']}")
    print(f"🕐 时间: {row['created_at']}")
    print("-"*40)

💬 消息总数: 1

📝 消息列表：

🆔 消息ID: 1
👤 用户: test1 (ID: 2)
💬 内容: 长江长江，我是黄河
🕐 时间: 2026-01-20 08:25:00.397130
----------------------------------------


In [8]:
# 表格形式展示消息
print("📊 消息表格视图：")
messages_df

📊 消息表格视图：


,id,content,user_id,username,created_at
0,1,长江长江，我是黄河,2,test1,2026-01-20 08:25:00.397130


## 4️⃣ 统计信息

---

In [9]:
# 统计每个用户的发消息数量
query = """
SELECT 
    u.username,
    COUNT(m.id) as message_count
FROM users u
LEFT JOIN messages m ON u.id = m.user_id
GROUP BY u.id
ORDER BY message_count DESC;
"""

user_stats = pd.read_sql_query(query, conn)

print("📊 用户发消息统计：")
print(user_stats.to_string(index=False))

📊 用户发消息统计：
username  message_count
   test1              1
testuser              0
   test2              0


In [10]:
# 每日消息数量统计
query = """
SELECT 
    DATE(created_at) as date,
    COUNT(*) as message_count
FROM messages
GROUP BY DATE(created_at)
ORDER BY date;
"""

daily_stats = pd.read_sql_query(query, conn)

print("📅 每日消息统计：")
print(daily_stats.to_string(index=False))

📅 每日消息统计：
      date  message_count
2026-01-20              1


## 5️⃣ 搜索功能

---

In [ ]:
# 搜索特定用户的消息
def search_user_messages(username):
    """搜索指定用户的所有消息"""
    query = """
    SELECT 
        m.id,
        m.content,
        m.created_at
    FROM messages m
    JOIN users u ON m.user_id = u.id
    WHERE u.username = ?
    ORDER BY m.created_at DESC;
    """
    
    result = pd.read_sql_query(query, conn, params=(username,))
    return result

# 使用示例：搜索用户名包含 "test" 的消息
username_to_search = "test"
print(f"🔍 搜索用户 '{username_to_search}' 的消息：\n")

result = search_user_messages(username_to_search)
if len(result) > 0:
    print(result.to_string(index=False))
else:
    print("未找到该用户的消息")

In [ ]:
# 搜索包含特定关键词的消息
def search_messages(keyword):
    """搜索包含关键词的消息"""
    query = """
    SELECT 
        m.id,
        u.username,
        m.content,
        m.created_at
    FROM messages m
    JOIN users u ON m.user_id = u.id
    WHERE m.content LIKE ?
    ORDER BY m.created_at DESC;
    """
    
    result = pd.read_sql_query(query, conn, params=(f"%{keyword}%",))
    return result

# 使用示例：搜索包含 "hello" 的消息
keyword_to_search = "hello"
print(f"🔍 搜索包含 '{keyword_to_search}' 的消息：\n")

result = search_messages(keyword_to_search)
if len(result) > 0:
    print(result.to_string(index=False))
else:
    print("未找到包含该关键词的消息")

## 6️⃣ 高级查询

---

In [ ]:
# 获取最新 N 条消息
def get_latest_messages(limit=10):
    """获取最新 N 条消息"""
    query = """
    SELECT 
        m.id,
        u.username,
        m.content,
        m.created_at
    FROM messages m
    JOIN users u ON m.user_id = u.id
    ORDER BY m.created_at DESC
    LIMIT ?;
    """
    
    result = pd.read_sql_query(query, conn, params=(limit,))
    return result

# 获取最新 5 条消息
print("📝 最新 5 条消息：\n")
latest = get_latest_messages(5)
print(latest.to_string(index=False))

In [ ]:
# 获取用户发消息的时间分布
def get_user_activity_hours(username):
    """获取用户发消息的小时分布"""
    query = """
    SELECT 
        CAST(strftime('%H', created_at) AS INTEGER) as hour,
        COUNT(*) as message_count
    FROM messages m
    JOIN users u ON m.user_id = u.id
    WHERE u.username = ?
    GROUP BY hour
    ORDER BY hour;
    """
    
    result = pd.read_sql_query(query, conn, params=(username,))
    return result

# 查看某个用户的活跃时间
print("⏰ 用户发消息时间分布：\n")
activity = get_user_activity_hours("testuser")
if len(activity) > 0:
    print(activity.to_string(index=False))
else:
    print("未找到该用户的消息")

## 🔐 密码安全检查

---

In [ ]:
# 验证密码是否正确
from passlib.context import CryptContext

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

def verify_password(username, plain_password):
    """验证用户密码"""
    query = """
    SELECT hashed_password FROM users WHERE username = ?;
    """
    
    result = pd.read_sql_query(query, conn, params=(username,))
    
    if len(result) == 0:
        return False, "用户不存在"
    
    hashed_password = result.iloc[0]['hashed_password']
    
    if pwd_context.verify(plain_password, hashed_password):
        return True, "密码正确"
    else:
        return False, "密码错误"

# 测试密码验证
print("🔐 密码验证测试：\n")

test_cases = [
    ("testuser", "testpass"),
    ("testuser", "wrongpass"),
    ("nonexistent", "pass"),
]

for username, password in test_cases:
    success, message = verify_password(username, password)
    status = "✅" if success else "❌"
    print(f"{status} 用户: {username}, 密码: {password} → {message}")

In [ ]:
# 检查密码强度（简单检查）
print("🔒 密码强度分析：\n")

query = """
SELECT username, hashed_password FROM users;
"""

users = pd.read_sql_query(query, conn)

for idx, row in users.iterrows():
    pwd_len = len(row['hashed_password'])
    print(f"👤 {row['username']}:")
    print(f"   密码哈希长度: {pwd_len} 字符")
    print(f"   哈希算法: bcrypt (通过 passlib)")
    print(f"   安全等级: ✅ 高 (bcrypt 自动加盐)")
    print()

## 📊 数据导出

---

In [ ]:
# 导出用户数据到 CSV
def export_users_to_csv(filename="users_export.csv"):
    """导出用户数据到 CSV 文件"""
    query = "SELECT * FROM users;"
    df = pd.read_sql_query(query, conn)
    df.to_csv(filename, index=False)
    print(f"✅ 用户数据已导出到: {filename}")

# 导出消息数据到 CSV
def export_messages_to_csv(filename="messages_export.csv"):
    """导出消息数据到 CSV 文件"""
    query = """
    SELECT 
        m.id,
        m.content,
        m.user_id,
        u.username,
        m.created_at
    FROM messages m
    JOIN users u ON m.user_id = u.id;
    """
    df = pd.read_sql_query(query, conn)
    df.to_csv(filename, index=False)
    print(f"✅ 消息数据已导出到: {filename}")

# 执行导出
export_users_to_csv()
export_messages_to_csv()

## ❌ 关闭连接

---

In [ ]:
# 关闭数据库连接
conn.close()
print("🔒 数据库连接已关闭")